# Join sector timeseries into one file

When running Timeseries_sectors.py the timeseries for each sector is saved as a separate file.
Here we create a new dataset with each sector as a variable, and save as one file.

In [1]:
import os
import glob
import xarray as xr

from acrg.name import name

In [2]:
site = 'WAO'
year = '2018'

### Get the separate timeseries files

In [3]:
ts_path = os.path.join('/user', 'work', 'vf20487', 'Timeseries', 'o2_co2')
ts_files = glob.glob(os.path.join(ts_path, f'{site}_ff-ukghg-*_timeseries_{year}.nc'))
ts_files = {ts_file.split('ukghg-')[1].split('_time')[0]: ts_file
            for ts_file in ts_files}
ts_files

{'indproc': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-indproc_timeseries_2018.nc',
 'energyprod': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-energyprod_timeseries_2018.nc',
 'roadtrans': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-roadtrans_timeseries_2018.nc',
 'othertrans': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-othertrans_timeseries_2018.nc',
 'offshore': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-offshore_timeseries_2018.nc',
 'domcom': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-domcom_timeseries_2018.nc',
 'indcom': '/user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-indcom_timeseries_2018.nc'}

### Join the data

Open the files to get the timeseries for each sector, rename to add the sector into the variable name, and combine into one dataset

In [4]:
ts_data = {sector: name.open_ds(ts_file) for sector, ts_file in ts_files.items()}
ts_data = {sector: ts.rename({'o2': f'o2-{sector}',
                              'co2': f'co2-{sector}'})
           for sector, ts in ts_data.items()}
ts_data_all = xr.merge(ts_data.values())

In [5]:
ts_data_all

<xarray.Dataset>
Dimensions:         (time: 8760)
Coordinates:
  * time            (time) datetime64[ns] 2018-01-01 ... 2018-12-31T23:00:00
Data variables: (12/14)
    o2-indproc      (time) float64 -0.01304 -0.0007964 ... -0.003724 -0.005437
    co2-indproc     (time) float64 0.01582 0.000966 ... 0.004512 0.006587
    o2-energyprod   (time) float64 -0.8403 -0.9682 -0.8509 ... -2.703 -2.967
    co2-energyprod  (time) float64 0.5055 0.5824 0.5111 ... 1.569 1.612 1.769
    o2-roadtrans    (time) float64 -0.5335 -0.3745 -0.3436 ... -1.435 -1.6
    co2-roadtrans   (time) float64 0.3703 0.2599 0.2384 ... 1.017 0.9961 1.11
    ...              ...
    o2-offshore     (time) float64 -0.0008622 -0.0008735 ... -0.001644 -0.0019
    co2-offshore    (time) float64 0.0005832 0.0005904 ... 0.001105 0.001276
    o2-domcom       (time) float64 -0.958 -0.9379 -0.9336 ... -1.591 -1.803
    co2-domcom      (time) float64 0.5089 0.4982 0.4959 ... 0.8516 0.8441 0.9564
    o2-indcom       (time) float64 -0.3105 -0.3034 -0.3603 ... -0.8854 -1.064
    co2-indcom      (time) float64 0.1885 0.1841 0.2185 ... 0.4181 0.533 0.6408

### Save

In [6]:
filename = os.path.join(ts_path, f'{site}_ff-ukghg-sectors_timeseries_{year}.nc')
print(f'Saving to {filename}')
ts_data_all.to_netcdf(filename)

Saving to /user/work/vf20487/Timeseries/o2_co2/WAO_ff-ukghg-sectors_timeseries_2018.nc


### Remove separate files

In [7]:
remove_sep_files = [os.remove(ts_file) for ts_file in ts_files.values()]